<a href="https://colab.research.google.com/github/mjmousavi97/Deep-Learning-Tehran-uni/blob/main/HomeWorks/06%20HW/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

from sklearn.metrics import accuracy_score, f1_score, precision_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),

    # transform to adding noise
    transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.1),

    transforms.RandomRotation((-20, 20)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# downsample = nn.Sequential(
#     nn.Conv2d(3, 256, kernel_size=1, stride=1, bias=False),
#     nn.BatchNorm2d(256)
# )

In [ ]:
class bottleneck(nn.Module):
    class Bottleneck(nn.Module):
    """
    Bottleneck block used in ResNet-50/101/152.

    This block has three convolutional layers:
      1. 1x1 conv to reduce channels.
      2. 3x3 conv for spatial processing.
      3. 1x1 conv to expand channels by the expansion factor (default 4).

    A residual (shortcut) connection is added from input to output.
    If the input and output shapes don't match, a `downsample` layer can
    be provided to adjust them.

    Parameters
    ----------
    in_channels : int
        Number of channels in the input.
    out_channels : int
        Number of channels in the bottleneck layers. Output will be out_channels * 4.
    stride : int, optional
        Stride for the 3x3 conv. Defaults to 1.
    downsample : nn.Module or None, optional
        Layer to match the input to output shape if needed.

    Notes
    -----
    Use downsample when stride > 1 or the number of channels changes.
    """

    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()

        self.stride = stride
        self.downsample = downsample
        self.relu = nn.ReLU(inplace=True)

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = self.BatchNorm2d(out_channels)

        self.conv3 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels*self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)


    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, in_channels=64, layers, num_classes=1000):

        super().__init__()

        self.in_channels = in_channels
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)


    def forward(self, x):
        pass